# Задача 2
## В таблице приведены данные о содержании иммуноглобулина Ig A в сыворотке крови у больных пяти возрастных групп:
| № | Содержание Ig A(%) |
| ----------- | ----------- |
| 1 | 83, 85 |
| 2 | 84, 85, 85, 86, 86, 87 |
| 3 | 86, 87, 87, 87, 88, 88, 88, 88, 88, 89, 90 |
| 4 | 89, 90, 90, 91 |
| 5 | 90, 92 |

## a) Определить влияние возраста на содержание иммуноглобулина в крови с помощью регрессионного анализа.

In [133]:
import numpy as np
from rich.console import Console
from rich.table import Table
from scipy.stats import t, f, norm
import numpy as np
from scipy import stats

alpha = 0.05
console = Console()

age_groups = {
    1: [[1, 1] for i in range(2)],
    2: [[1, 2] for i in range(6)],
    3: [[1, 3] for i in range(11)],
    4: [[1, 4] for i in range(4)],
    5: [[1, 5] for i in range(2)],
}

PSI = []
PSI.extend(age_groups[1])
PSI.extend(age_groups[2])
PSI.extend(age_groups[3])
PSI.extend(age_groups[4])
PSI.extend(age_groups[5])
PSI = np.array(PSI)

ig_age_groups = {
    1: [83, 85],
    2: [84, 85, 85, 86, 86, 87],
    3: [86, 87, 87, 87, 88, 88, 88, 88, 88, 89, 90],
    4: [89, 90, 90, 91],
    5: [90, 92],
}

Y = []
Y.extend(ig_age_groups[1])
Y.extend(ig_age_groups[2])
Y.extend(ig_age_groups[3])
Y.extend(ig_age_groups[4])
Y.extend(ig_age_groups[5])
Y = np.array(Y)

F = np.dot(PSI.transpose(), PSI)
F_inverse = np.linalg.inv(F)

beta_wave = np.dot(np.dot(F_inverse, PSI.transpose()), Y)

n = len(PSI)
p = len(beta_wave)

table = Table(show_header=True, header_style="green")
table.add_column("Коэффициент", justify="center")
table.add_column("Значение", justify="center")
table.add_column("p-value", justify="center")
table.add_column("Значим/Незначим", justify="center")

e = np.array(Y - np.dot(PSI, beta_wave))

RSS = np.dot(e.transpose(), e)

for i, beta_i in enumerate(beta_wave):
    delta_wave = abs(beta_i / np.sqrt(RSS * F_inverse[i][i]) * np.sqrt(n - p))
    p_value = 2 * t.sf(delta_wave, n - p)
    table.add_row(
        f"β{i}",
        f"{beta_i:.3f}",
        f"{p_value}",
        f"{'Значим' if p_value < alpha else 'Незначим'}",
    )

console.print(table)

┏━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Коэффициент ┃ Значение ┃        p-value         ┃ Значим/Незначим ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│     β0      │  81.896  │ 2.0140201783267344e-34 │     Значим      │
│     β1      │  1.940   │ 2.1251512362036984e-09 │     Значим      │
└─────────────┴──────────┴────────────────────────┴─────────────────┘

### Уравнение регрессии: $${y = 81.896 + 1.940 * x_1},$$ где $x_1$ равен номеру возрастной группы.

In [134]:
TSS = sum((np.mean(Y) - Y) ** 2)

delta_wave = ((TSS - RSS) / (p - 1)) / (RSS / (n - p))

p_value = f.sf(delta_wave, p - 1, n - p)

console.print(f"p-value = {p_value} {'>' if p_value > alpha else '<'} {alpha}")

R_squared = (TSS - RSS) / TSS

console.print(f"R² = {R_squared}")

p-value = 2.125151236203109e-09 < 0.05

R² = 0.7957938733346712

### Так как p-value меньше 0.05, то регрессия значима.

## b) Провести попарное сравнение средних в рамках регрессионной модели, с учетом множественности проверяемых гипотез.

In [139]:
all_pairs = [(i, j) for i in range(1, 6) for j in range(i + 1, 6)]

n = len(all_pairs)

table = Table(show_header=True, header_style="green")
table.add_column("Первая группа", justify="center")
table.add_column("Вторая группа", justify="center")
table.add_column("α", justify="center")
table.add_column("p-value", justify="center")
table.add_column("Значимо/незначимо различие средних между группами", justify="center")

all_p_values = dict()
for pair in all_pairs:
    t_stat, p_value = stats.ttest_ind(
        ig_age_groups[pair[0]], ig_age_groups[pair[1]], equal_var=True
    )
    if p_value < alpha:
        all_p_values[(pair[0], pair[1])] = p_value
    else:
        table.add_row(f"{pair[0]}", f"{pair[1]}", f"{alpha}", f"{p_value}", f"Незначимо")

m = len(all_p_values)
for pair, p_value in sorted(all_p_values.items(), key=lambda item: item[1]):
    alpha_i = alpha / m
    if p_value < alpha_i:
        m -= 1
        table.add_row(f"{pair[0]}", f"{pair[1]}", f"{alpha_i}", f"{p_value}", f"Значимо")
    else:
        table.add_row(f"{pair[0]}", f"{pair[1]}", f"{alpha_i}", f"{p_value}", f"Незначимо")


console.print(table)

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃               ┃               ┃                       ┃                       ┃   Значимо/незначимо различие    ┃
┃ Первая группа ┃ Вторая группа ┃           α           ┃        p-value        ┃     средних между группами      ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       1       │       2       │         0.05          │   0.15145400164755    │            Незначимо            │
│       4       │       5       │         0.05          │  0.31250000000000006  │            Незначимо            │
│       2       │       4       │        0.00625        │ 9.24505788450397e-05  │             Значимо             │
│       2       │       3       │ 0.0071428571428571435 │ 0.0006664488749708028 │             Значимо             │
│       2       │       5       │ 0.008333333333333333  │ 0.0009437516449898814 │             Значимо             │
│       1       │       3       │         0.01          │ 0.0009608874579818152 │             Значимо             │
│       1       │       4       │        0.0125         │ 0.0022784262913801683 │             Значимо             │
│       3       │       4       │ 0.016666666666666666  │ 0.0029454107549331853 │             Значимо             │
│       3       │       5       │         0.025         │ 0.0033959610991523056 │             Значимо             │
│       1       │       5       │         0.05          │ 0.038476052359176824  │             Значимо             │
└───────────────┴───────────────┴───────────────────────┴───────────────────────┴─────────────────────────────────┘